# `SENTIMENTAL ANALYSIS`

#### `IMPORTIN LIBRARIES`

In [21]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from textblob import Word
import pickle
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ganes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### `IMPORTING DATASET`

In [22]:
ds=pd.read_csv("tweet_train.csv",encoding="ISO-8859-1",names=["label", "ids", "date", "flag", "user", "tweet"])
ds=ds.drop(["ids","date","flag","user"],axis=1)

In [23]:
positive=ds[ds["label"]==0][:20000]
neagtive=ds[ds["label"]==4][:20000]

In [24]:
positive.info()
print()
neagtive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   20000 non-null  int64 
 1   tweet   20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 468.8+ KB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 800000 to 819999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   20000 non-null  int64 
 1   tweet   20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 468.8+ KB


In [25]:
positive

,label,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
19995,0,"Not much time off this weekend, work trip to M..."
19996,0,One more day of holidays
19997,0,feeling so down right now .. i hate you DAMN H...
19998,0,"geez,i hv to READ the whole book of personalit..."


In [26]:
neagtive["label"]=neagtive["label"].apply(lambda x:1)
neagtive

,label,tweet
800000,1,I LOVE @Health4UandPets u guys r the best!!
800001,1,im meeting up with one of my besties tonight! ...
800002,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,1,Being sick can be really cheap when it hurts t...
800004,1,@LovesBrooklyn2 he has that effect on everyone
...,...,...
819995,1,We just met some awesome people at T.G.I Fridays
819996,1,oh my goodness . my feet are the most sensitiv...
819997,1,@hey_angy hahaha
819998,1,@Lamartian30 Pfft school is koo. I'm ranked 2n...


In [27]:
dataset=pd.concat([positive,neagtive])
dataset=dataset.sample(frac=1)
dataset

,label,tweet
816838,1,finish practising pipa
810400,1,@MacDavid hey david!!! thanks for your kind wo...
813615,1,Just realized that ALL of my blip songs sudden...
810592,1,Strange but wonderful. Actually very strange ...
803977,1,@geekgirls No problem!
...,...,...
10618,0,nasa US na sis ko... learning how to be indep...
801114,1,"ok so the twilight book is pretty addicting, c..."
3306,0,stooopppiddd Abbey National!! i need to get do...
811424,1,@brentkworth i just need it


### `PRE-PROCESSING`

##### `PRE PROCESSING VARIABLE SETTING`

In [28]:
# dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
dataset['tweet'] = dataset['tweet'].apply(lambda x: re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+",' ',x))

In [29]:
dataset

,label,tweet
816838,1,finish practising pipa
810400,1,hey david thanks for your kind words about m...
813615,1,Just realized that ALL of my blip songs sudden...
810592,1,Strange but wonderful Actually very strange
803977,1,No problem
...,...,...
10618,0,nasa US na sis ko learning how to be independe...
801114,1,ok so the twilight book is pretty addicting ch...
3306,0,stooopppiddd Abbey National i need to get dosh...
811424,1,i just need it


In [30]:
dataset["tweet"]=dataset["tweet"].apply(lambda x: " ".join([w for w in x.split() if w not in stopwords.words("english")]))
dataset

,label,tweet
816838,1,finish practising pipa
810400,1,hey david thanks kind words site
813615,1,Just realized ALL blip songs suddenly r unavai...
810592,1,Strange wonderful Actually strange
803977,1,No problem
...,...,...
10618,0,nasa US na sis ko learning independent hahahah...
801114,1,ok twilight book pretty addicting chapter 6 ti...
3306,0,stooopppiddd Abbey National need get dosh bloo...
811424,1,need


In [31]:
# Lemmatizing

dataset['tweet']=dataset['tweet'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
dataset

,label,tweet
816838,1,finish practising pipa
810400,1,hey david thanks kind word site
813615,1,Just realized ALL blip song suddenly r unavail...
810592,1,Strange wonderful Actually strange
803977,1,No problem
...,...,...
10618,0,nasa US na si ko learning independent hahahaha...
801114,1,ok twilight book pretty addicting chapter 6 ti...
3306,0,stooopppiddd Abbey National need get dosh bloo...
811424,1,need


### `SPLITTING DATA FOR TRAINING AND TESTING`

#### `LSTM MODEL`

In [32]:
max_words = 2000

def tokenize_pad_sequences(text):
    tokenizer = Tokenizer(num_words=max_words, split=' ')
    tokenizer.fit_on_texts(text)
    X = tokenizer.texts_to_sequences(text)
    X = pad_sequences(X)
    return X, tokenizer

print('Before Tokenization & Padding \n', dataset['tweet'][0])
X, tokenizer = tokenize_pad_sequences(dataset['tweet'])
print('After Tokenization & Padding \n', X[0])

Before Tokenization & Padding 
 Awww bummer You shoulda got David Carr Third Day D
After Tokenization & Padding 
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0 575]


In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [33]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_words, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 28, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
from sklearn.model_selection import train_test_split
Y=pd.get_dummies(dataset["label"]).values
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(32000, 28) (32000, 2)
(8000, 28) (8000, 2)


In [35]:
batch_size = 32
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
1000/1000 - 54s - loss: 0.5561 - accuracy: 0.7111 - 54s/epoch - 54ms/step
Epoch 2/7
1000/1000 - 54s - loss: 0.4983 - accuracy: 0.7544 - 54s/epoch - 54ms/step
Epoch 3/7
1000/1000 - 54s - loss: 0.4758 - accuracy: 0.7697 - 54s/epoch - 54ms/step
Epoch 4/7
1000/1000 - 55s - loss: 0.4578 - accuracy: 0.7772 - 55s/epoch - 55ms/step
Epoch 5/7
1000/1000 - 61s - loss: 0.4385 - accuracy: 0.7878 - 61s/epoch - 61ms/step
Epoch 6/7
1000/1000 - 60s - loss: 0.4244 - accuracy: 0.7950 - 60s/epoch - 60ms/step
Epoch 7/7
1000/1000 - 90s - loss: 0.4067 - accuracy: 0.8033 - 90s/epoch - 90ms/step


In [48]:
twt = ['so good to be bad']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0  79   3 303 895  58]]
1/1 - 0s - 22ms/epoch - 22ms/step
positive


In [46]:
model.evaluate(X_test,y_test)

250/250 [==============================] - 4s 16ms/step - loss: 0.5874 - accuracy: 0.7234


[0.5874378085136414, 0.7233750224113464]

In [49]:
model.save("lstm_model.h5")